스파크 세선을 성성하고 버전을 확인합니다.
상세한 가이드는 Spark Streaming + Kafka Integration Guide (Kafka broker version 0.10.0 or higher) - Spark 3.2.1 Documentation 를 참고 합니다.

In [1]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
from IPython.display import display, display_pretty, clear_output, JSON

spark = (
    SparkSession
    .builder
    .config("spark.sql.session.timeZone", "Asia/Seoul")
    .getOrCreate()
)

# 노트북에서 테이블 형태로 데이터 프레임 출력을 위한 설정을 합니다
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # display enabled
spark.conf.set("spark.sql.repl.eagerEval.truncate", 100) # display output columns size

# 공통 데이터 위치
home_jovyan = "/home/jovyan"
work_data = f"{home_jovyan}/work/data"
work_dir=!pwd
work_dir = work_dir[0]

# 로컬 환경 최적화
spark.conf.set("spark.sql.shuffle.partitions", 5) # the number of partitions to use when shuffling data for joins or aggregations.
spark.conf.set("spark.sql.streaming.forceDeleteTempCheckpointLocation", "true")

# 현재 기동된 스파크 애플리케이션의 포트를 확인하기 위해 스파크 정보를 출력합니다
spark

노트북에서 스파크 스트리밍 상태 및 데이터 조회를 위한 함수 선언

In [2]:
# 스트림 테이블을 주기적으로 조회하는 함수 (name: 이름, sql: Spark SQL, iterations: 반복횟수, sleep_secs: 인터벌)
# 기존 출력 cell을 지우고, 이름, 현재 반복 횟수, 현재 수행 쿼리를 출력합니다.
# 쿼리 결과를 출력하고 sleep sec초 만큼 대기하는 과정을 iteration 번 반복해서 출력합니다.
def displayStream(name, sql, iterations, sleep_secs):
    from time import sleep
    i = 1
    for x in range(iterations):
        clear_output(wait=True)              # 출력 Cell clear
        display('[' + name + '] Iteration: '+str(i)+', Query: '+sql)
        display(spark.sql(sql))              # Spark SQL 수행
        sleep(sleep_secs)                    # sleep_secs 초 만큼 대기
        i += 1

# 스트림 쿼리의 상태를 주기적으로 조회하는 함수 (name: 이름, query: Streaming Query, iterations: 반복횟수, sleep_secs: 인터벌)
def displayStatus(name, query, iterations, sleep_secs):
    from time import sleep
    i = 1
    for x in range(iterations):
        clear_output(wait=True)      # Output Cell 의 내용을 지웁니다
        display('[' + name + '] Iteration: '+str(i)+', Status: '+query.status['message'])
        display(query.lastProgress)  # 마지막 수행된 쿼리의 상태를 출력합니다
        sleep(sleep_secs)            # 지정된 시간(초)을 대기합니다
        i += 1

카프카로부터 메시지 수신을 위한 카프카 리더를 생성합니다
earliest 옵션으로 kafka:9093 의 movies 토픽으로 부터 메시지를 읽어와서 데이터 변환을 수행합니다.

In [9]:
# kafkaReader 생성 (spark.readStream) -> 카프카에서 메세지 수신
# server.properties > advertised.listeners=INSIDE://kafka:9093,OUTSIDE://localhost:9092

kafkaReader = (
    spark
  .readStream
  .format("kafka")
  .option("kafka.bootstrap.servers", "kafka:9093")
  .option("subscribe", "movies")
  .option("startingOffsets", "earliest")
  .load()
)
kafkaReader.printSchema()


root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [ ]:
들어오는 데이터 스키마를 명시적으로 정의해줍니다.
KafkaSelector 를 추가해서 카프카에서 읽어들인 데이터(KafkaReader)에서 key 컬럼 추가하고

In [16]:
kafkaSchema = (
    StructType()
    .add(StructField("status", StringType()))
    .add(StructField("id", StringType()))
    .add(StructField("screen_name", StringType()))
    .add(StructField("user_id", IntegerType()))
    .add(StructField("profile", StringType()))
    .add(StructField("time", StringType()))
    .add(StructField("text", StringType()))
    .add(StructField("retweet_count", IntegerType()))
    .add(StructField("favorite_count", IntegerType()))
    .add(StructField("lat", IntegerType()))
    .add(StructField("long", IntegerType()))
)

kafkaSelector = (
    kafkaReader
    .select(
        col("key").cast("string"),
        from_json(col("value").cast("string"), kafkaSchema).alias("movies")
    )
    .selectExpr("movies.id as key", "to_json(struct(movies.*)) as value")
)

kafkaSelector.printSchema()

root
 |-- key: string (nullable = true)
 |-- value: string (nullable = true)



디버깅을 위해 임시로 콘솔 출력을 통해 검증합니다
스트리밍 데이터는 디버깅이 상당히 어렵기 때문에 새로운 카프카 토픽 혹은 외부에 저장하기 전에 반드시 눈으로 확인해야 합니다

위에서 선언한 displayStream 함수를 활용하여 메시지를 읽어서 출력합니다.

In [14]:
# 노트북 로그 콘솔로 출력
queryName = "consoleSink3"
kafkaWriter = (
    kafkaSelector.select("key", "value")
    .writeStream
    .queryName(queryName)
    .format("memory")
    .outputMode("append")
)

checkpointLocation = f"{work_dir}/tmp/{queryName}"
!rm -rf $checkpointLocation

kafkaTrigger = (
    kafkaWriter
    .trigger(processingTime="5 second")
    .option("checkpointLocation", checkpointLocation)
)

# 파이썬의 경우 콘솔 디버깅이 노트북 표준출력으로 나오기 때문에, 별도 메모리 테이블로 조회
kafkaQuery = kafkaTrigger.start()
displayStream(queryName, f"select * from {queryName} order by key desc", 4, 5)
kafkaQuery.stop()

'[consoleSink3] Iteration: 4, Query: select * from consoleSink3 order by key desc'

key,value
1582347914386341889,"{""status"":""ORIGINAL"",""id"":""1582347914386341889"",""screen_name"":""rkiveonfilm"",""profile"":""http://pbs..."
1582347913153302528,"{""status"":""ORIGINAL"",""id"":""1582347913153302528"",""screen_name"":""kookiebunnyjjk7"",""profile"":""http:/..."
1582347911660072961,"{""status"":""ORIGINAL"",""id"":""1582347911660072961"",""screen_name"":""eriykcal"",""profile"":""http://pbs.tw..."
1582347910762483712,"{""status"":""ORIGINAL"",""id"":""1582347910762483712"",""screen_name"":""BTSJKSJH"",""profile"":""http://pbs.tw..."
1582347910716739585,"{""status"":""ORIGINAL"",""id"":""1582347910716739585"",""screen_name"":""minsaybt"",""profile"":""http://pbs.tw..."
1582347910322475008,"{""status"":""ORIGINAL"",""id"":""1582347910322475008"",""screen_name"":""supewgemoww"",""profile"":""http://pbs..."
1582347910150094848,"{""status"":""ORIGINAL"",""id"":""1582347910150094848"",""screen_name"":""taetaecutiefufu"",""profile"":""http:/..."
1582347909726863360,"{""status"":""ORIGINAL"",""id"":""1582347909726863360"",""screen_name"":""Army_ipurpleu09"",""profile"":""http:/..."
1582347908791164928,"{""status"":""ORIGINAL"",""id"":""1582347908791164928"",""screen_name"":""MIMI_KOKO13"",""profile"":""http://pbs..."
1582347908690518016,"{""status"":""ORIGINAL"",""id"":""1582347908690518016"",""screen_name"":""Aesthetixxx9"",""profile"":""http://pb..."


카프카 movies 토픽을 새로운 토픽 korean_movies 로 출력 합니다
예제에서는 거의 원본 그대로를 출력하지만, 실제로는 Dimension 테이블과 Join 을 하거나, 데이터 가공 및 변환 등의 Enrich 단계를 수행하며, 우리 예제 에서는 kafkaSelect 부분만 수정하면 됩니다.

새로운 토픽 korean_movies 으로 전송을 위해 kafkaWriter 코드만 다시 작성합니다. 출력 모드는 반드시 append 모드로 수행하여

In [ ]:
# 카프카로 다시 저장
queryName = "kafkaSink"
kafkaWriter = (
    kafkaSelector.select("key", "value")
    .writeStream
    .queryName(queryName)
    .format("kafka")
    .option("kafka.bootstrap.servers", "카프카주소:카프카포트")
    .option("topic", "저장대상카프카토픽")
    .outputMode("출력모드")
)

checkpointLocation = f"{work_dir}/tmp/{queryName}"
!rm -rf $checkpointLocation

kafkaTrigger = (
    kafkaWriter
    .trigger(processingTime="5 seconds")
    .option("checkpointLocation", checkpointLocation)
)

kafkaQuery = kafkaTrigger.start()
displayStatus(queryName, kafkaQuery, 1000, 10)
kafkaQuery.stop()